In [1]:
# Course DSC 650 - Data Mining
# Name - Vikas Ranjan
# Assignment - Assignment 11 - Text Generation with LSTM

### Text generation with LSTM 
Since we need is a lot of text data that we can use to learn a language model. You could use any suﬀiciently large text file or set of text files – Wikipedia, the Lord of the Rings, etc. In this example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model we will learn will thus be specifically a model of Nietzsche’s writing style and topics of choice, rather than a more generic model of the English language.

In [3]:
import keras 
keras.__version__
import numpy as np
from keras import layers
import random
import sys
from keras import layers

Download the nietzsche file and convert the contents to lowercase.

In [4]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 1us/step
Corpus length: 600893


Extract partially-overlapping sequences of length maxlen, one-hot encode them and pack them in a 3D Numpy array x of shape (sequences, maxlen, unique_characters). Simultane- ously, we prepare a array y containing the corresponding targets: the one-hot encoded characters that come right after each extracted sequence.

In [5]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step): 
    sentences.append(text[i: i + maxlen]) 
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

# Dictionary mapping unique characters to their index in `chars` 
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) 
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence): 
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


### Building the network
Our network is a single LSTM layer followed by a Dense classifier and softmax over all possible characters. But let us note that recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in recent times.

In [6]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [7]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

### Training the language model and sampling from it
Given a trained model and a seed text snippet, we generate new text by repeatedly:
1) Drawing from the model a probability distribution over the next character given the text available so far
2) Reweighting the distribution to a certain “temperature”
3) Sampling the next character at random according to the reweighted distribution
4) Adding the new character at the end of the available text
This is the code we use to reweight the original probability distribution coming out of the model, and draw a character index from it (the “sampling function”):

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64') 
    preds = np.log(preds) / temperature 
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds) 
    probas = np.random.multinomial(1, preds, 1) 
    return np.argmax(probas)

Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of temperature in the sampling strategy.

In [9]:
for epoch in range(1, 5):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data 
    model.fit(x, y,
              batch_size=128,
              epochs=1)
    
    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]: 
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars))) 
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
    
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            generated_text += next_char
            generated_text = generated_text[1:]
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
1565/1565 [==============================] - 199s 125ms/step - loss: 2.2675
--- Generating with seed: "ious
conscience, that the self-reliance of the community is "
------ temperature: 0.2
ious
conscience, that the self-reliance of the community is the ore in the man in the mest and sould been an in the read and in the really of the experience of the reading that who is the experience of the most than the highest than the most and in the most than the most than the experience of the experience of the most and all the man and are that which is the most the for the reading and be sould the most than the most that the philosophy that the experi
------ temperature: 0.5
d the most than the most that the philosophy that the experience, when in the denerally than the ens that is dessent the power in the more that is of man of the sill and or should the experience, the readous that the possible that the thright of the viel, world be in the explaments that is the man of the and the be
o